In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.6 MB/s eta 0:00:00


In [2]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [3]:
PROJECT_ID = "qwiklabs-gcp-01-df3e58eb0cfc"
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [27]:
model_name = "gemini-2.0-flash-001"
model = GenerativeModel(model_name)

## Define the output format & specify constraints

In [5]:
transcript = """
    Speaker 1 (Customer): Hi, can I get a cheeseburger and large fries, please?
    Speaker 2 (Restaurant employee): Coming right up! Anything else you'd like to add to your order?
    Speaker 1: Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?
    Speaker 2: No problem, one cheeseburger, one large fries with ketchup on the side, and a small
    orange juice. That'll be $5.87. Drive through to the next window please.
"""

In [6]:
response = model.generate_content(f"""
    Extract the transcript to JSON.

    {transcript}
""")

print(response.text)

```json
[
  {
    "speaker": "Customer",
    "utterance": "Hi, can I get a cheeseburger and large fries, please?"
  },
  {
    "speaker": "Restaurant employee",
    "utterance": "Coming right up! Anything else you'd like to add to your order?"
  },
  {
    "speaker": "Customer",
    "utterance": "Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?"
  },
  {
    "speaker": "Restaurant employee",
    "utterance": "No problem, one cheeseburger, one large fries with ketchup on the side, and a small orange juice. That'll be $5.87. Drive through to the next window please."
  }
]
```


In [7]:
response = model.generate_content(f"""
    <INSTRUCTIONS>
    - Extract the ordered items into JSON.
    - Separate drinks from food.
    - Include a quantity for each item and a size if specified.
    </INSTRUCTIONS>

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
""")

print(response.text)

```json
{
  "food_items": [
    {
      "item": "cheeseburger",
      "quantity": 1
    },
    {
      "item": "fries",
      "quantity": 1,
      "size": "large",
      "special_instructions": "ketchup on the side"
    }
  ],
  "drink_items": [
    {
      "item": "orange juice",
      "quantity": 1,
      "size": "small"
    }
  ]
}
```


## Assign a persona or role

In [8]:
chat = model.start_chat()

In [9]:
response = chat.send_message(
    """
    Provide a brief guide to caring for the houseplant monstera deliciosa?
    """
)

print(response.text)

## Monstera Deliciosa Care: A Quick Guide

The Monstera Deliciosa, also known as the Swiss Cheese Plant, is a relatively easy and rewarding houseplant to care for. Here's a brief guide:

**Light:**

*   **Bright, indirect light is ideal.** Avoid direct sunlight which can scorch the leaves. An east or west-facing window is usually a good spot.

**Water:**

*   **Water when the top 1-2 inches of soil are dry.** Stick your finger in the soil to check.
*   **Water thoroughly** until water drains out of the bottom of the pot.
*   **Avoid overwatering!** This can lead to root rot.

**Humidity:**

*   **Monsteras enjoy humidity.** Mist the leaves occasionally, use a humidifier, or place the plant on a pebble tray filled with water.

**Soil:**

*   **Well-draining potting mix is essential.** A mix of potting soil, perlite, and orchid bark works well.

**Temperature:**

*   **Average room temperature (65-80°F / 18-27°C) is fine.** Avoid extreme temperatures.

**Fertilizer:**

*   **Fertilize du

In [10]:
new_chat = model.start_chat()

response = new_chat.send_message(
    """
    You are a houseplant monstera deliciosa. Help the person who
    is taking care of you to understand your needs.
    """
)

print(response.text)

Okay, friend! Let's talk. It's me, your Monstera Deliciosa, or as you might affectionately call me, your Monstera! I appreciate you trying to keep me happy. Here's a little insight into what makes me thrive:

**Light is Everything (Well, Almost Everything):**

*   **Bright, indirect light is my sweet spot.** Think of it like being near a sunny window, but with a sheer curtain filtering the harsh direct rays. Direct sunlight can scorch my beautiful leaves! If you see faded patches or sunburn marks, I'm getting too much direct sun.
*   **Not enough light is no good either.** If my growth is slow, my leaves are small and dark green, and I'm not developing any of those cool splits and holes (fenestrations), I need more light. Consider moving me closer to a window or using a grow light.

**Watering Wisely (Don't Drown Me!)**

*   **Let me dry out a bit between waterings.** Stick your finger about an inch or two into the soil. If it feels dry, it's time to water. If it feels moist, hold off.

## Include examples

In [11]:
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Example Message: My department has been using a vendor for
our development, and we are interested in exploring other
options. Do you have time for a discussion around your
services?
Likelihood: 2

Example Message: I have mockups drawn for an app and a budget
allocated. We are interested in moving forward to have a
proof of concept built within 2 months, with plans to develop
it further in the following quarter.
Likelihood: 3

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)

3



## Experiment with parameter values

In [12]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature": 0.05}
)

print(response.text)

Why did the frog call his insurance company?

Because he got toad!



In [13]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .98,
                       "temperature": 1}
)

print(response.text)

Why did the frog cross the road?

To get to the hop-posite side!



## Utilize fallback responses

In [14]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: How high can a horse jump?
    """
)

print(response.text)

Sorry, I only talk about pottery!



In [15]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: What is the difference between ceramic
    and porcelain? Please keep your response brief.
    """
)

print(response.text)

Ceramic is a general term for pottery made from clay and hardened by heat. Porcelain is a specific type of ceramic made from fine clay and fired at high temperatures, resulting in a translucent and delicate material.



## Add contextual information

In [16]:
response = model.generate_content(
    """
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

I do not have access to real-time inventory information or specific store layouts. To find the aisle numbers for those items, you'll need to:

*   **Check the store's website or app:** Many grocery stores have websites or apps that allow you to search for items and see their aisle location.
*   **Use an in-store kiosk:** Some stores have kiosks where you can search for products and get directions.
*   **Ask a store employee:** The easiest way to find out is to ask a store employee. They will be able to quickly tell you the correct aisle.


In [17]:
response = model.generate_content("""
    Context:
    Michael's Grocery Store Aisle Layout:
    Aisle 1: Fruits — Apples, bananas,  grapes, oranges, strawberries, avocados, peaches, etc.
    Aisle 2: Vegetables — Potatoes, onions, carrots, salad greens, broccoli, peppers, tomatoes, cucumbers, etc.
    Aisle 3: Canned Goods — Soup, tuna, fruit, beans, vegetables, pasta sauce, etc.
    Aisle 4: Dairy — Butter, cheese, eggs, milk, yogurt, etc.
    Aisle 5: Meat— Chicken, beef, pork, sausage, bacon etc.
    Aisle 6: Fish & Seafood— Shrimp, crab, cod, tuna, salmon, etc.
    Aisle 7: Deli— Cheese, salami, ham, turkey, etc.
    Aisle 8: Condiments & Spices— Black pepper, oregano, cinnamon, sugar, olive oil, ketchup, mayonnaise, etc.
    Aisle 9: Snacks— Chips, pretzels, popcorn, crackers, nuts, etc.
    Aisle 10: Bread & Bakery— Bread, tortillas, pies, muffins, bagels, cookies, etc.
    Aisle 11: Beverages— Coffee, teabags, milk, juice, soda, beer, wine, etc.
    Aisle 12: Pasta, Rice & Cereal—Oats, granola, brown rice, white rice, macaroni, noodles, etc.
    Aisle 13: Baking— Flour, powdered sugar, baking powder, cocoa etc.
    Aisle 14: Frozen Foods — Pizza, fish, potatoes, ready meals, ice cream, etc.
    Aisle 15: Personal Care— Shampoo, conditioner, deodorant, toothpaste, dental floss, etc.
    Aisle 16: Health Care— Saline, band-aid, cleaning alcohol, pain killers, antacids, etc.
    Aisle 17: Household & Cleaning Supplies—Laundry detergent, dish soap, dishwashing liquid, paper towels, tissues, trash bags, aluminum foil, zip bags, etc.
    Aisle 18: Baby Items— Baby food, diapers, wet wipes, lotion, etc.
    Aisle 19: Pet Care— Pet food, kitty litter, chew toys, pet treats, pet shampoo, etc.

    Query:
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

Here's where you can find those items at Michael's Grocery Store based on the provided aisle layout:

*   **paper plates:** Aisle 17 (Household & Cleaning Supplies)
*   **mustard:** Aisle 8 (Condiments & Spices)
*   **potatoes:** Aisle 2 (Vegetables) and Aisle 14 (Frozen Foods)



## Structure prompts with prefixes or tags

In [18]:
prompt = """
  <OBJECTIVE_AND_PERSONA>
  You are a dating matchmaker.
  Your task is to identify common topics or interests between
  the USER_ATTRIBUTES and POTENTIAL_MATCH options and present them
  as a fun and meaningful potential matches.
  </OBJECTIVE_AND_PERSONA>

  <INSTRUCTIONS>
  To complete the task, you need to follow these steps:
  1. Identify matching or complimentary elements from the
     USER_ATTRIBUTES and the POTENTIAL_MATCH options.
  2. Pick the POTENTIAL_MATCH that represents the best match to the USER_ATTRIBUTES
  3. Describe that POTENTIAL_MATCH like an encouraging friend who has
     found a good dating prospect for a friend.
  4. Don't insult the user or potential matches.
  5. Only mention the best match. Don't mention the other potential matches.
  </INSTRUCTIONS>

  <CONTEXT>
  <USER_ATTRIBUTES>
  Name: Allison
  I like to go to classical music concerts and the theatre.
  I like to swim.
  I don't like sports.
  My favorite cuisines are Italian and ramen. Anything with noodles!
  </USER_ATTRIBUTES>

  <POTENTIAL_MATCH 1>
  Name: Jason
  I'm very into sports.
  My favorite team is the Detroit Lions.
  I like baked potatoes.
  </POTENTIAL_MATCH 1>

  <POTENTIAL_MATCH 2>
  Name: Felix
  I'm very into Beethoven.
  I like German food. I make a good spaetzle, which is like a German pasta.
  I used to play water polo and still love going to the beach.
  </POTENTIAL_MATCH 2>
  </CONTEXT>

  <OUTPUT_FORMAT>
  Format results in Markdown.
  </OUTPUT_FORMAT>
"""

response = model.generate_content(prompt)

print(response.text)

Okay, Allison, listen to this! I think I've found someone you might really click with.

Let me tell you about Felix. He is really into Beethoven, just like you enjoy classical music! And guess what? He makes spaetzle, which is a *German pasta*! Noodles! Plus, he used to play water polo, so he likes to swim and hang out at the beach! I think you two would have a lot to talk about and enjoy doing together!



## Use system instructions

In [28]:
system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel(model_name,
                    system_instruction=system_instructions)

response = music_model.generate_content(
    """
    Who is worth studying?
    """
)

print(response.text)

There are countless musical figures worth studying, each offering unique insights into the art form. Let's explore a few diverse examples:

*   **Johann Sebastian Bach (Baroque Period):** A master of counterpoint and harmony, Bach's works (e.g., "The Well-Tempered Clavier," "Mass in B Minor") demonstrate unparalleled structural ingenuity and emotional depth. Studying Bach provides a foundation for understanding Western music theory and composition.

*   **Ludwig van Beethoven (Classical/Romantic Periods):** Bridging the Classical and Romantic eras, Beethoven's symphonies, sonatas, and string quartets revolutionized musical form and expression. His personal struggles and innovative spirit make him a compelling figure. For example, his Symphony No. 5 is one of the most recognizable and analyzed pieces in Western music.

*   **Bessie Smith (Blues):** Known as the "Empress of the Blues," Smith's powerful vocals and emotionally charged performances (e.g., "Downhearted Blues," "Nobody Knows 

## Demonstrate Chain-of-Thought

In [22]:
question = """
Instructions:
Use the context and make any updates needed in the scenario to answer the question.

Context:
A high efficiency factory produces 100 units per day.
A medium efficiency factory produces 60 units per day.
A low efficiency factory produces 30 units per day.

Megacorp owns 5 factories. 3 are high efficiency, 2 are low efficiency.

<EXAMPLE SCENARIO>
Scenario:
Tomorrow Megacorp will have to shut down one high efficiency factory.
It will add two rented medium efficiency factories to make up production.

Question:
How many units can they produce today? How many tomorrow?

Answer:

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 2 factories * 100 units/day/factory = 200 units/day
* Medium efficiency factories: 2 factories * 60 units/day/factory = 120 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 380 units/day**
</EXAMPLE SCENARIO>

<SCENARIO>
Scenario:
Tomorrow Megacorp will reconfigure a low efficiency factory up to medium efficiency.
And the remaining low efficiency factory has an outage that cuts output in half.

Question:
How many units can they produce today? How many tomorrow?

Answer: """

response = model.generate_content(question,
                                  generation_config={"temperature": 0})
print(response.text)

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Medium efficiency factories: 1 factory * 60 units/day/factory = 60 units/day
* Low efficiency factories: 1 factory * (30 units/day/factory / 2) = 15 units/day
* **Total production tomorrow: 300 units/day + 60 units/day + 15 units/day = 375 units/day**



## Break down complex tasks

In [23]:
response = model.generate_content(
    """
    To explain the difference between a TPU and a GPU, what are
    five different ideas for metaphors that compare the two?
    """
)

brainstorm_response = response.text
print(brainstorm_response)

Okay, here are five different metaphors to explain the difference between TPUs (Tensor Processing Units) and GPUs (Graphics Processing Units):

1.  **Construction Crew Metaphor:**

    *   **GPU:**  Imagine a general construction crew. They are good at a variety of tasks: framing, plumbing, electrical, drywalling, etc. They can build a house from start to finish, adapting to different needs along the way.  They can do a little of everything pretty well, even if it takes some time to switch between tasks.

    *   **TPU:**  Now imagine a specialized prefabrication plant for constructing houses.  Instead of building homes from scratch, it mass produces prefabricated walls or rooms with everything already installed. It's *extremely* efficient at making those specific pre-built parts, but not very good at anything else. It excels in repetitive and consistent tasks.

    *   **Relevance:** GPUs are general-purpose and can handle a wide array of computational tasks. TPUs are specialized for 

In [24]:
response = model.generate_content(
    """
    From the perspective of a college student learning about
    computers, choose only one of the following explanations
    of the difference between TPUs and GPUs that captures
    your visual imagination while contributing
    to your understanding of the technologies.

    {brainstorm_response}
    """.format(brainstorm_response=brainstorm_response)
)

student_response = response.text

print(student_response)

Okay, of those five, I choose **the Formula 1 Pit Crew Metaphor.** Here's why:

*   **Visual Imagination:** The image of a Formula 1 pit crew is instantly engaging and visually stimulating. The quick, coordinated movements, the bright colors, and the sense of urgency are all appealing. I can easily picture the difference between a general-purpose crew fumbling with different tasks and a highly specialized crew performing a single task with laser focus.

*   **Contribution to Understanding:** The metaphor clearly highlights the core difference between GPUs and TPUs: specialization vs. versatility. A standard pit crew is like a GPU; they can handle a variety of tasks during a pit stop (refueling, tire changes, minor repairs). However, a TPU is like a pit crew *solely* dedicated to changing tires. They've optimized every movement and process to do that one task faster than anyone else. This perfectly illustrates how TPUs are designed for the repetitive, parallel computations common in mac

In [25]:
response = model.generate_content(
    """
    Elaborate on the choice of metaphor below by turning
    it into an introductory paragraph for a blog post.

    {student_response}
    """.format(student_response=student_response)
)

blog_post = response.text

print(blog_post)

Ever watched a Formula 1 pit stop? The controlled chaos, the lightning-fast movements, the laser-focused team members all working in perfect synchronicity? That image perfectly encapsulates the difference between GPUs and TPUs, those powerful chips that drive the world of computing. While a general-purpose GPU is like a standard pit crew, capable of handling a variety of tasks from refueling to minor repairs, a TPU is akin to the highly specialized tire-changing team, meticulously trained and optimized to perform a single task faster than anyone else. Let's dive into how this analogy explains the core differences between these two processors and why specialization makes TPUs a force to be reckoned with in the world of machine learning.

